In [ ]:
# --- main_runner.py ---
# Main orchestration script for running the ML asset pricing pipeline.
# Imports config and utils, defines model training logic, runs the pipeline loops.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import time
import traceback
from collections import defaultdict
import random
import re

# --- Import Configuration & Utilities ---
import config
import pipeline_utils as utils

# --- Import Model Specific Libraries ---
from sklearn.linear_model import LinearRegression, ElasticNetCV, ElasticNet, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid, KFold
from sklearn.metrics import mean_squared_error

try:
    import statsmodels.api as sm
    STATSMODELS_AVAILABLE = True
except ImportError: STATSMODELS_AVAILABLE = False; print("ADVARSEL: Statsmodels ikke funnet.")
try:
    # Ensure TF is imported correctly
    import tensorflow as tf
    tf.config.set_visible_devices([], 'GPU') # Explicitly disable GPU if causing issues, or manage memory growth
    from tensorflow import keras
    from tensorflow.keras import layers, regularizers, callbacks, backend as K
    from tensorflow.keras.optimizers import Adam # Use legacy Adam if needed: from tensorflow.keras.optimizers.legacy import Adam

    TENSORFLOW_AVAILABLE = True
    print(f"TensorFlow version: {tf.__version__}")
    # Set TF seeds and deterministic options
    os.environ['PYTHONHASHSEED']=str(config.TF_SEED)
    os.environ['TF_DETERMINISTIC_OPS'] = '1' # Newer TF versions might use this
    os.environ['TF_CUDNN_DETERMINISTIC']='1' # Note: May impact performance
    random.seed(config.TF_SEED)
    np.random.seed(config.TF_SEED)
    tf.random.set_seed(config.TF_SEED)
    # Optional: Configure GPU memory growth if using GPU
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently no GPUs are available. Uncomment if you have GPUs.
            # tf.config.set_visible_devices(gpus[0], 'GPU') # Select specific GPU if needed
            # for gpu in gpus:
            #     tf.config.experimental.set_memory_growth(gpu, True)
            print(f"GPUs funnet ({len(gpus)}), konfigurasjon forsøkt (men muligens deaktivert ovenfor).")
        except RuntimeError as e: print(f"Kunne ikke sette minnevekst for GPU: {e}")
    else:
        print("Ingen GPU funnet, bruker CPU.")

except ImportError: TENSORFLOW_AVAILABLE = False; print("ADVARSEL: TensorFlow/Keras ikke funnet.")

# ==========================================================================
# --- MODEL TRAINING/EVALUATION FUNCTIONS (Per Window) ---
# (Keep OLS, OLS3H, PLS, PCR, ENET, GLM_H, RF, GBRT_H exactly as before)
# Define OLS
def train_evaluate_ols(X_train_val, y_train_val, X_test, model_params):
    try:
        model = LinearRegression(fit_intercept=True).fit(X_train_val, y_train_val)
        preds_oos = model.predict(X_test) if X_test.shape[0] > 0 else np.array([])
        preds_is = model.predict(X_train_val)
        return model, preds_oos, preds_is, {}
    except Exception as e: print(f"    FEIL OLS: {e}"); return None, np.array([]), np.array([]), {}

# Define OLS3H
def train_evaluate_ols3h(X_train_val, y_train_val, X_test, model_params):
    if not STATSMODELS_AVAILABLE: return None, np.array([]), np.array([]), {}
    try:
        X_tv_const = sm.add_constant(X_train_val, prepend=True, has_constant='add') # Ensure constant is added correctly
        X_test_const = sm.add_constant(X_test, prepend=True, has_constant='add') if X_test.shape[0] > 0 else None
        rlm_model = sm.RLM(y_train_val, X_tv_const, M=sm.robust.norms.HuberT())
        valid_fit_params = {k: v for k, v in model_params.items() if k in ['maxiter', 'tol']}
        fitted_model = rlm_model.fit(**valid_fit_params)
        preds_oos = fitted_model.predict(X_test_const) if X_test_const is not None else np.array([])
        preds_is = fitted_model.predict(X_tv_const)
        # Store actual params used, including Huber M
        optim_params = {'M': 'HuberT', **valid_fit_params}
        return fitted_model, preds_oos, preds_is, optim_params
    except Exception as e: print(f"    FEIL OLS3H: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}


# Define _tune_simple_model helper
def _tune_simple_model(ModelClass, X_train, y_train, X_val, y_val, param_grid_dict):
    best_mse = np.inf; best_param_value = None; param_name = list(param_grid_dict.keys())[0]
    param_values = param_grid_dict[param_name]
    # Adjust max components check for safety
    max_components = min(X_train.shape[0], X_train.shape[1])
    valid_grid = [p for p in param_values if isinstance(p, int) and 0 < p <= max_components]
    if not valid_grid:
        print(f"    Advarsel: Ingen gyldig grid for {param_name} (max={max_components}). Bruker 1 komponent.")
        valid_grid = [1] # Fallback to 1 component if grid is invalid

    for p_val in valid_grid:
        try:
            if ModelClass == Pipeline: # For PCR
                 model_val = Pipeline([('pca', PCA(n_components=p_val)), ('lr', LinearRegression())])
            else: # For PLS
                 model_val = ModelClass(**{param_name: p_val, 'scale': False})

            model_val.fit(X_train, y_train)
            y_pred_val = model_val.predict(X_val).flatten()
            if not np.all(np.isfinite(y_pred_val)): continue # Skip if predictions are not finite

            mse = mean_squared_error(y_val, y_pred_val)
            if not np.isnan(mse) and mse < best_mse:
                 best_mse = mse; best_param_value = p_val

        except Exception as e:
            # print(f"    FEIL tuning {ModelClass.__name__} {param_name}={p_val}: {e}") # Optional debug
            continue # Skip parameter if fitting fails

    if best_param_value is None: print(f"    FEIL: Tuning mislyktes for {ModelClass.__name__}.")
    return best_param_value


# Define PLS
def train_evaluate_pls(X_train, y_train, X_val, y_val, X_test, model_params):
    optimal_params = {}
    try:
        best_n = _tune_simple_model(PLSRegression, X_train, y_train, X_val, y_val, {'n_components': model_params['n_components_grid']})
        if best_n is None: raise ValueError("PLS tuning failed.")
        optimal_params = {'n_components': best_n}
        X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
        final_model = PLSRegression(n_components=best_n, scale=False).fit(X_train_val, y_train_val)
        preds_oos = final_model.predict(X_test).flatten() if X_test.shape[0] > 0 else np.array([])
        preds_is = final_model.predict(X_train_val).flatten()
        return final_model, preds_oos, preds_is, optimal_params
    except Exception as e: print(f"    FEIL PLS: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}

# Define PCR
def train_evaluate_pcr(X_train, y_train, X_val, y_val, X_test, model_params):
    optimal_params = {}
    try:
        best_n = _tune_simple_model(Pipeline, X_train, y_train, X_val, y_val, {'n_components': model_params['n_components_grid']})
        if best_n is None: raise ValueError("PCR tuning failed.")
        optimal_params = {'n_components': best_n}
        final_model = Pipeline([('pca', PCA(n_components=best_n)), ('lr', LinearRegression())])
        X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
        final_model.fit(X_train_val, y_train_val)
        preds_oos = final_model.predict(X_test) if X_test.shape[0] > 0 else np.array([])
        preds_is = final_model.predict(X_train_val)
        return final_model, preds_oos, preds_is, optimal_params
    except Exception as e: print(f"    FEIL PCR: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}

# Define ENET
def train_evaluate_enet(X_train, y_train, X_test, model_params):
    optimal_params = {}
    try:
        cv_strategy = KFold(n_splits=model_params['cv_folds'], shuffle=True, random_state=config.GENERAL_SEED)
        # Use ElasticNetCV on the combined train+val set if tuning is desired across both
        # If tuning ONLY on train set (as paper might imply for simpler models?), fit only on X_train
        # Let's stick to fitting CV on X_train as per the original code structure for ENET
        enet_cv = ElasticNetCV(
            alphas=model_params['alphas'],
            l1_ratio=model_params['l1_ratio'],
            fit_intercept=True,
            cv=cv_strategy,
            n_jobs=model_params.get('n_jobs', -1),
            max_iter=model_params.get('max_iter', 1000),
            tol=model_params.get('tol', 0.001),
            random_state=config.GENERAL_SEED
        )
        enet_cv.fit(X_train, y_train) # Fit CV on training data only
        optimal_params = {'alpha': enet_cv.alpha_, 'l1_ratio': enet_cv.l1_ratio_}

        # Final model is the one found by CV
        final_model = enet_cv
        preds_oos = final_model.predict(X_test) if X_test.shape[0] > 0 else np.array([])
        preds_is = final_model.predict(X_train) # IS predictions on the data used for CV fitting
        return final_model, preds_oos, preds_is, optimal_params
    except Exception as e: print(f"    FEIL ENET: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}

# Define GLM_H
def train_evaluate_glm_h(X_train, y_train, X_val, y_val, X_test, model_params):
    optimal_params = {}; best_mse = np.inf; optim_found_params = None
    grid = list(ParameterGrid(model_params['param_grid'])); max_iter = model_params.get('max_iter', 300)
    # Base parameters for HuberRegressor
    base_hub_params = {'fit_intercept': True, 'max_iter': max_iter, 'tol': 1e-6} # Add tol

    for params in grid:
        try:
            # Combine base params with grid params
            current_params = {**base_hub_params, **params}
            model_val = HuberRegressor(**current_params).fit(X_train, y_train)
            y_pred_val = model_val.predict(X_val)
            if not np.all(np.isfinite(y_pred_val)): continue
            mse = mean_squared_error(y_val, y_pred_val)
            if not np.isnan(mse) and mse < best_mse:
                 best_mse = mse; optim_found_params = params # Store only the tuned params
        except Exception as e:
            # print(f"    FEIL GLM_H tuning params {params}: {e}") # Optional debug
            continue
    if optim_found_params is None: print("    FEIL: GLM_H tuning mislyktes."); return None, np.array([]), np.array([]), {}
    optimal_params = optim_found_params.copy()
    try:
        X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
        # Combine base params with optimal tuned params for final fit
        final_hub_params = {**base_hub_params, **optimal_params}
        final_model = HuberRegressor(**final_hub_params).fit(X_train_val, y_train_val)
        preds_oos = final_model.predict(X_test) if X_test.shape[0] > 0 else np.array([])
        preds_is = final_model.predict(X_train_val)
        return final_model, preds_oos, preds_is, optimal_params
    except Exception as e: print(f"    FEIL GLM_H final: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}

# Define _tune_tree_model helper
def _tune_tree_model(ModelClass, X_train, y_train, X_val, y_val, model_params):
    best_mse = np.inf; best_params = None; param_grid = list(ParameterGrid(model_params['param_grid']))
    # Base parameters are those NOT in the grid (e.g., n_jobs, random_state, loss for GBRT)
    base_params = {k: v for k, v in model_params.items() if k != 'param_grid'}
    for params in param_grid:
        try:
            current_params = {**base_params, **params}
            # Ensure max_features is valid if passed as float
            if 'max_features' in current_params and isinstance(current_params['max_features'], float):
                 current_params['max_features'] = max(1, int(current_params['max_features'] * X_train.shape[1]))

            model_val = ModelClass(**current_params).fit(X_train, y_train)
            y_pred_val = model_val.predict(X_val)
            if not np.all(np.isfinite(y_pred_val)): continue
            mse = mean_squared_error(y_val, y_pred_val)
            if not np.isnan(mse) and mse < best_mse:
                 best_mse = mse; best_params = params # Store only tuned params
        except Exception as e:
            # print(f"    FEIL {ModelClass.__name__} tuning params {params}: {e}") # Optional debug
            continue
    if best_params is None: print(f"    FEIL: Tuning mislyktes for {ModelClass.__name__}.")
    return best_params

# Define RF
def train_evaluate_rf(X_train, y_train, X_val, y_val, X_test, model_params):
    optimal_params = {}
    try:
        best_grid_params = _tune_tree_model(RandomForestRegressor, X_train, y_train, X_val, y_val, model_params)
        if best_grid_params is None: raise ValueError("RF tuning failed.")
        optimal_params = best_grid_params.copy()
        # Combine fixed params with optimal grid params
        final_params = {**{k:v for k,v in model_params.items() if k!='param_grid'}, **optimal_params}
        # Ensure max_features is valid if passed as float
        if 'max_features' in final_params and isinstance(final_params['max_features'], float):
             final_params['max_features'] = max(1, int(final_params['max_features'] * X_train.shape[1]))

        X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
        final_model = RandomForestRegressor(**final_params).fit(X_train_val, y_train_val)
        preds_oos = final_model.predict(X_test) if X_test.shape[0] > 0 else np.array([])
        preds_is = final_model.predict(X_train_val)
        return final_model, preds_oos, preds_is, optimal_params
    except Exception as e: print(f"    FEIL RF: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}

# Define GBRT_H
def train_evaluate_gbrt_h(X_train, y_train, X_val, y_val, X_test, model_params):
    optimal_params = {}
    try:
        # Ensure 'loss' is set correctly from config, not overridden by grid search
        gbrt_params = model_params.copy()
        gbrt_params['loss'] = config.MODEL_PARAMS['GBRT_H'].get('loss', 'huber') # Explicitly set loss
        best_grid_params = _tune_tree_model(GradientBoostingRegressor, X_train, y_train, X_val, y_val, gbrt_params)
        if best_grid_params is None: raise ValueError("GBRT tuning failed.")
        optimal_params = best_grid_params.copy()
        # Combine fixed params with optimal grid params
        final_params = {**{k:v for k,v in gbrt_params.items() if k!='param_grid'}, **optimal_params}

        X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
        final_model = GradientBoostingRegressor(**final_params).fit(X_train_val, y_train_val)
        preds_oos = final_model.predict(X_test) if X_test.shape[0] > 0 else np.array([])
        preds_is = final_model.predict(X_train_val)
        # Return only the tuned grid params as optimal
        return final_model, preds_oos, preds_is, optimal_params
    except Exception as e: print(f"    FEIL GBRT: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}

# --- Define NN functions (if TF available) ---
if TENSORFLOW_AVAILABLE:
    def build_nn_model(input_shape, nn_config, lambda1):
        model = keras.Sequential(name=nn_config['name'])
        model.add(layers.Input(shape=(input_shape,)))
        # Add Batch Normalization before the first hidden layer's activation
        model.add(layers.BatchNormalization())
        for units in nn_config['hidden_units']:
            model.add(layers.Dense(units, kernel_regularizer=regularizers.l1(lambda1)))
            # Add Batch Normalization after Dense layer, before activation
            model.add(layers.BatchNormalization())
            model.add(layers.Activation('relu')) # Use ReLU activation
        model.add(layers.Dense(1, activation='linear')) # Linear output layer
        return model

    # Generic NN training function
    def train_evaluate_nn(X_train, y_train, X_val, y_val, X_test, nn_shared_params, nn_specific_config):
        model_name = nn_specific_config['name']; optimal_params = {}; best_val_mse = np.inf; optim_found_params = None
        input_shape = X_train.shape[1]
        param_grid = list(ParameterGrid(nn_shared_params['param_grid']))
        epochs = nn_shared_params['epochs']; batch_size = nn_shared_params['batch_size']
        patience = nn_shared_params['patience']; ensemble_size = nn_shared_params['ensemble_size']
        base_seed = nn_shared_params['random_seed_base']

        print(f"      Tuning {model_name} ({len(param_grid)} combos)...")
        # Tuning Loop
        for params in param_grid:
            lambda1 = params['lambda1']; learning_rate = params['learning_rate']; val_preds_ensemble = []
            current_val_mses = [] # Track MSE for each member in ensemble for this param set
            try:
                for i in range(ensemble_size):
                    K.clear_session();
                    member_seed = base_seed + i + random.randint(0, 10000) # Add more randomness
                    tf.random.set_seed(member_seed); np.random.seed(member_seed); random.seed(member_seed)

                    nn_model = build_nn_model(input_shape, nn_specific_config, lambda1)
                    nn_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse') # Mean Squared Error loss
                    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True, verbose=0, mode='min')
                    history = nn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping, callbacks.TerminateOnNaN()], verbose=0)

                    # Check if training was successful and val_loss exists
                    if 'val_loss' in history.history and history.history['val_loss']:
                         best_epoch_val_loss = min(history.history['val_loss'])
                         if not np.isnan(best_epoch_val_loss):
                              current_val_mses.append(best_epoch_val_loss)
                              # Get predictions from the best weights restored by EarlyStopping
                              val_preds_member = nn_model.predict(X_val, batch_size=batch_size, verbose=0).flatten()
                              if np.all(np.isfinite(val_preds_member)):
                                   val_preds_ensemble.append(val_preds_member)
                              else:
                                   # print(f"      Warn: Non-finite val preds in ensemble member {i+1} for params {params}");
                                   val_preds_ensemble = []; break # Discard whole ensemble if one fails
                         else:
                             # print(f"      Warn: NaN val_loss in ensemble member {i+1} for params {params}")
                             val_preds_ensemble = []; break
                    else:
                         # print(f"      Warn: Training stopped early or val_loss missing for params {params}")
                         val_preds_ensemble = []; break

                if not val_preds_ensemble: continue # Skip if ensemble failed

                # Evaluate based on average prediction MSE or average member MSE
                # Using average prediction MSE:
                avg_val_preds = np.mean(np.array(val_preds_ensemble), axis=0)
                finite_mask = np.isfinite(avg_val_preds) & np.isfinite(y_val)
                if np.sum(finite_mask) > 0:
                     val_mse = mean_squared_error(y_val[finite_mask], avg_val_preds[finite_mask])
                     if not np.isnan(val_mse) and val_mse < best_val_mse:
                          best_val_mse = val_mse; optim_found_params = params
                          # print(f"      New best params for {model_name}: {params} (MSE: {val_mse:.6f})")
                # else: print(f"      Warn: No finite overlapping preds/targets for params {params}")

            except Exception as e:
                print(f"    FEIL NN tuning {model_name} params {params}: {e}"); traceback.print_exc(limit=1); continue

        # Check Tuning Success
        if optim_found_params is None: print(f"    FEIL: NN tuning mislyktes for {model_name}."); return None, np.array([]), np.array([]), {}

        optimal_params = optim_found_params.copy(); opt_lambda1 = optimal_params['lambda1']; opt_lr = optimal_params['learning_rate']
        print(f"      Optimal params for {model_name}: {optimal_params}")

        # Final Ensemble Training on Train + Val
        final_model = None; test_preds_ensemble = []; is_preds_ensemble = []
        try:
            X_train_val = np.vstack((X_train, X_val)); y_train_val = np.concatenate((y_train, y_val))
            print(f"      Training final {model_name} ensemble ({ensemble_size} members)...")
            for i in range(ensemble_size):
                K.clear_session();
                final_seed = base_seed + i + ensemble_size + random.randint(0, 10000) # Different seeds for final run
                tf.random.set_seed(final_seed); np.random.seed(final_seed); random.seed(final_seed)

                nn_model_final = build_nn_model(input_shape, nn_specific_config, opt_lambda1)
                nn_model_final.compile(optimizer=Adam(learning_rate=opt_lr), loss='mse')
                # No early stopping for final model, train for full epochs
                history_final = nn_model_final.fit(X_train_val, y_train_val, epochs=epochs, batch_size=batch_size, callbacks=[callbacks.TerminateOnNaN()], verbose=0)

                if 'loss' not in history_final.history or not history_final.history['loss'] or np.isnan(history_final.history['loss']).any():
                     print(f"      FEIL: NaN loss during final training member {i+1} for {model_name}")
                     test_preds_ensemble = []; is_preds_ensemble = []; break # Discard ensemble

                # Store IS and OOS predictions for this member
                preds_i = nn_model_final.predict(X_train_val, batch_size=batch_size, verbose=0).flatten()
                if np.all(np.isfinite(preds_i)): is_preds_ensemble.append(preds_i)
                else: print(f"      FEIL: Non-finite IS preds final member {i+1} for {model_name}"); is_preds_ensemble = []; break

                if X_test.shape[0] > 0:
                    preds_t = nn_model_final.predict(X_test, batch_size=batch_size, verbose=0).flatten()
                    if np.all(np.isfinite(preds_t)): test_preds_ensemble.append(preds_t)
                    else: print(f"      FEIL: Non-finite OOS preds final member {i+1} for {model_name}"); test_preds_ensemble = []; break

                # Keep the last trained model instance (for potential inspection, although ensemble average is used)
                if i == ensemble_size - 1: final_model = nn_model_final

            # Aggregate Predictions if ensemble finished successfully
            if (X_test.shape[0] > 0 and len(test_preds_ensemble) != ensemble_size) or len(is_preds_ensemble) != ensemble_size:
                 print(f"    FEIL: {model_name} final ensemble failed or did not complete.")
                 return None, np.array([]), np.array([]), {}

            preds_oos_final = np.mean(np.array(test_preds_ensemble), axis=0) if X_test.shape[0] > 0 else np.array([])
            preds_is_final = np.mean(np.array(is_preds_ensemble), axis=0)
            return final_model, preds_oos_final, preds_is_final, optimal_params
        except Exception as e:
            print(f"    FEIL NN final training/prediction for {model_name}: {e}"); traceback.print_exc(limit=1); return None, np.array([]), np.array([]), {}
else:
     print("TensorFlow ikke tilgjengelig, hopper over NN modeller.")
     # Define dummy functions if TF not available? Or rely on the check in main loop.
# ==========================================================================


# ==========================================================================
# --- MAIN EXECUTION SCRIPT ---
# ==========================================================================
if __name__ == "__main__":
    overall_start_time = datetime.datetime.now()
    print(f"--- Starter ML Asset Pricing Pipeline ---")
    print(f"--- Starttidspunkt: {overall_start_time:%Y-%m-%d %H:%M:%S} ---")
    print(f"--- Output lagres i: {config.OUTPUT_DIR} ---")

    # === 1: Load Preprocessed Data ===
    df_loaded = utils.load_prepare_data(
        config.DATA_FILE, config.COLUMN_CONFIG,
        config.TARGET_VARIABLE, config.NEXT_RETURN_VARIABLE, config.MARKET_CAP_ORIG_VARIABLE
    )
    if df_loaded is None: exit("Avslutter: Lasting av forhåndsbehandlet data feilet.")

    # === 2: Define Features (from preprocessed data) ===
    # *** Updated Exclusion List ***
    base_exclude_list = [
        config.TARGET_VARIABLE, config.NEXT_RETURN_VARIABLE, config.MARKET_CAP_ORIG_VARIABLE,
        'Instrument', 'Date', 'id', 'date', # Include potential original names
        'AdjustedReturn_t', 'MonthlyRiskFreeRate_t', # Exclude intermediate calcs if present
        # *** Explicitly exclude user requested columns and variants ***
        'MonthlyReturn_t', 'OpenPrice', 'monthlyreturn_t', 'openprice',
        'MonthlyReturn', 'monthlyreturn', # Add base names too
    ]
    # Add specific raw names to exclude if their log versions are intended features (dynamic check)
    # Check preprocess_data.py output for final log names (e.g., log_marketcap vs log_MarketCap)
    if utils.find_col(df_loaded, ['log_MarketCap', 'log_marketcap']): base_exclude_list.extend(utils.find_col(df_loaded, [n]) for n in ['MarketCap', 'CommonSharesOutstanding', 'ClosePrice'] if utils.find_col(df_loaded, [n]))
    if utils.find_col(df_loaded, ['log_BM', 'log_bm']): base_exclude_list.append(utils.find_col(df_loaded, ['BM', 'bm']))
    # if utils.find_col(df_loaded, ['log_ClosePrice', 'log_closeprice']): base_exclude_list.append(utils.find_col(df_loaded, ['ClosePrice', 'closeprice'])) # Already excluded if MarketCap logged
    if utils.find_col(df_loaded, ['log_Volume', 'log_volume']): base_exclude_list.append(utils.find_col(df_loaded, ['Volume', 'volume']))
    # if utils.find_col(df_loaded, ['log_CommonSharesOutstanding', 'log_commonsharesoutstanding']): base_exclude_list.append(utils.find_col(df_loaded, ['CommonSharesOutstanding', 'commonsharesoutstanding'])) # Already excluded if MarketCap logged
    if utils.find_col(df_loaded, ['TermSpread', 'termspread']): base_exclude_list.extend(utils.find_col(df_loaded, [n]) for n in ['NorgesBank10Y','NIBOR3M'] if utils.find_col(df_loaded, [n]))

    # Remove None values potentially added by find_col if column wasn't found
    base_exclude_list = sorted(list(set([col for col in base_exclude_list if col is not None])))
    print(f"INFO: Base columns to exclude from features: {base_exclude_list}")

    all_numeric_features_init, ols3_subset_features_init, _ = utils.define_features(
        df_loaded, config.OLS3_FEATURE_NAMES, base_exclude_list
    )
    if not all_numeric_features_init: exit("Avslutter: Ingen features definert etter lasting.")

    # === 3: Rank Standardize Features ===
    df_std = utils.rank_standardize_features(df_loaded, all_numeric_features_init)
    if df_std is None: exit("Avslutter: Standardisering feilet.")

    # === 4: Clean Data (Post-Standardization) ===
    df_clean = utils.clean_data(
        df_std,
        all_numeric_features_init, # Features to check for NaN/inf
        config.ESSENTIAL_COLS_FOR_DROPNA, # Columns where NaN forces row drop
        config.MARKET_CAP_ORIG_VARIABLE   # Column to check for > 0
    )
    if df_clean is None or df_clean.empty: exit("Avslutter: Dataframe tom etter rensing.")

    # === Final Feature Definition and Model Assignment ===
    all_numeric_features, ols3_subset_features, _ = utils.define_features(
        df_clean, config.OLS3_FEATURE_NAMES, base_exclude_list # Use same exclusion list
    )
    if not all_numeric_features: exit("FEIL: Ingen numeriske features igjen etter rensing.")

    ols3_required_count = len(config.OLS3_FEATURE_NAMES)
    if not ols3_subset_features or len(ols3_subset_features) < ols3_required_count:
        if config.RUN_MODELS.get('OLS3H', False): print(f"\nADVARSEL: Ikke alle {ols3_required_count} OLS3 features funnet ({ols3_subset_features}). OLS3H deaktiveres.")
        config.RUN_MODELS['OLS3H'] = False
    elif config.RUN_MODELS.get('OLS3H', False): print(f"\nINFO: Alle OLS3 features funnet ({ols3_subset_features}). OLS3H er aktiv.")

    feature_map = {}
    for model, fset_key in config.MODEL_FEATURE_MAP.items():
        if fset_key == 'ols3_features': feature_map[model] = ols3_subset_features if config.RUN_MODELS.get('OLS3H', False) else []
        elif fset_key == 'all_numeric': feature_map[model] = all_numeric_features
        else: print(f"Advarsel: Ukjent feature set key '{fset_key}' for {model}."); feature_map[model] = all_numeric_features

    # === Initialize Results Storage ===
    all_metrics = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    all_vi = defaultdict(lambda: defaultdict(list)); all_vi_avg = defaultdict(dict)
    all_portfolios = defaultdict(dict); all_summaries = {}

    # --- Define Actual Column Names (using config now) ---
    target_col_actual_name = config.TARGET_VARIABLE
    next_ret_col_actual_name = config.NEXT_RETURN_VARIABLE
    mcap_orig_col_actual_name = config.MARKET_CAP_ORIG_VARIABLE
    date_col_actual_name = 'Date'; id_col_actual_name = 'Instrument'

    essential_check_list = [date_col_actual_name, id_col_actual_name, target_col_actual_name, next_ret_col_actual_name, mcap_orig_col_actual_name]
    missing_essentials = [c for c in essential_check_list if c not in df_clean.columns]
    if missing_essentials: exit(f"FEIL: Nødvendige kolonner mangler i df_clean: {missing_essentials}.")

    # === 5: Outer Loop: Subsets ===
    print(f"\n--- Starter Subset Loop: {config.SUBSETS_TO_RUN} ---")
    for subset in config.SUBSETS_TO_RUN:
        subset_start_time = datetime.datetime.now()
        print(f"\n{'='*30} Starter Subset: {subset.upper()} {'='*30}")

        # --- Create Subset Data ---
        df_subset = pd.DataFrame()
        df_mc_temp = df_clean.dropna(subset=[date_col_actual_name, mcap_orig_col_actual_name]).copy()
        if df_mc_temp.empty: print(f"FEIL: Ingen data for subsetting i {subset}."); continue
        if subset == 'all': df_subset = df_clean.copy(); print("  Bruker fullt datasett.")
        else:
            print(f"  Definerer subset basert på {mcap_orig_col_actual_name} persentiler...")
            df_mc_temp['MonthYear'] = pd.to_datetime(df_mc_temp[date_col_actual_name]).dt.to_period('M')
            if subset == 'big':
                cutoff_quantile = 1.0 - (config.BIG_FIRM_TOP_PERCENT / 100.0)
                size_cutoffs = df_mc_temp.groupby('MonthYear')[mcap_orig_col_actual_name].quantile(cutoff_quantile)
                print(f"  -> Topp {config.BIG_FIRM_TOP_PERCENT}% (Quantile: {cutoff_quantile:.2f})")
                df_mc_temp = df_mc_temp.join(size_cutoffs.rename('cutoff'), on='MonthYear')
                # Handle cases where cutoff might be NaN (e.g., only one stock in a month)
                df_subset = df_mc_temp[(df_mc_temp[mcap_orig_col_actual_name] >= df_mc_temp['cutoff']) | df_mc_temp['cutoff'].isna()].copy()
            elif subset == 'small':
                cutoff_quantile = config.SMALL_FIRM_BOTTOM_PERCENT / 100.0
                size_cutoffs = df_mc_temp.groupby('MonthYear')[mcap_orig_col_actual_name].quantile(cutoff_quantile)
                print(f"  -> Bunn {config.SMALL_FIRM_BOTTOM_PERCENT}% (Quantile: {cutoff_quantile:.2f})")
                df_mc_temp = df_mc_temp.join(size_cutoffs.rename('cutoff'), on='MonthYear')
                # Handle cases where cutoff might be NaN
                df_subset = df_mc_temp[(df_mc_temp[mcap_orig_col_actual_name] <= df_mc_temp['cutoff']) | df_mc_temp['cutoff'].isna()].copy()

            else: print(f"FEIL: Ukjent subset '{subset}'."); continue
            df_subset = df_subset.drop(columns=['MonthYear', 'cutoff'], errors='ignore')
        if df_subset.empty: print(f"FEIL: Subset '{subset}' er tomt."); continue
        df_subset = df_subset.sort_values(by=[date_col_actual_name, id_col_actual_name]).reset_index(drop=True)
        print(f"  Subset '{subset}' klar. Form: {df_subset.shape}")

        # === 6: Inner Loop: Rolling Windows ===
        try: splits = list(utils.get_yearly_rolling_splits(df_subset, config.INITIAL_TRAIN_YEARS, config.VALIDATION_YEARS, config.TEST_YEARS_PER_WINDOW))
        except ValueError as e: print(f"FEIL split gen for '{subset}': {e}"); continue
        except Exception as e_gen: print(f"Uventet FEIL split gen for '{subset}': {e_gen}"); traceback.print_exc(); continue
        if not splits: print(f"Ingen vinduer for '{subset}'."); continue
        num_windows = len(splits)
        print(f"\n--- Starter Rullerende Vindu Loop for Subset: {subset} ({num_windows} vinduer) ---")
        window_preds_list = []; last_train_idx, last_val_idx = None, None; last_models_fit = {}

        for window_idx, (train_idx, val_idx, test_idx, train_dates, val_dates, test_dates) in enumerate(splits):
            window_num = window_idx + 1; window_start_time = time.time()
            print(f"\n-- Vindu {window_num}/{num_windows} ({subset}) --")
            if train_dates is not None: print(f"  Train: {train_dates['min'].date()} -> {train_dates['max'].date()} ({len(train_idx)} obs)")
            else: print("  Train: Tomt"); continue # Skip if no training data
            if val_dates is not None:   print(f"  Val:   {val_dates['min'].date()} -> {val_dates['max'].date()} ({len(val_idx)} obs)")
            else: print("  Val: Tomt") # Val might be empty for some models
            if test_dates is not None:  print(f"  Test:  {test_dates['min'].date()} -> {test_dates['max'].date()} ({len(test_idx)} obs)")
            else: print("  Test: Tomt"); continue # Skip if no test data

            if test_idx.empty or train_idx.empty: print("  Advarsel: Tomt train/test sett."); continue
            needs_val_set = lambda name: name not in ['OLS', 'OLS3H', 'ENET'] # Models needing validation set
            if val_idx.empty and any(config.RUN_MODELS[m] and needs_val_set(m) for m in config.RUN_MODELS if config.RUN_MODELS[m]):
                 print("  ADVARSEL: Tomt val set, men trengs av noen modeller. Hopper over disse modellene.");

            # Prepare data for this window
            y_train = df_subset.loc[train_idx, target_col_actual_name].values
            y_val = df_subset.loc[val_idx, target_col_actual_name].values if not val_idx.empty else np.array([])
            y_test = df_subset.loc[test_idx, target_col_actual_name].values
            y_train_val = np.concatenate((y_train, y_val)) if not val_idx.empty else y_train

            # Ensure targets are finite
            y_train_finite_mask = np.isfinite(y_train)
            y_train = y_train[y_train_finite_mask]
            train_idx = train_idx[y_train_finite_mask]
            if not val_idx.empty:
                y_val_finite_mask = np.isfinite(y_val)
                y_val = y_val[y_val_finite_mask]
                val_idx = val_idx[y_val_finite_mask]
            y_train_val = np.concatenate((y_train, y_val)) if not val_idx.empty else y_train


            window_results = {date_col_actual_name: df_subset.loc[test_idx, date_col_actual_name].values, id_col_actual_name: df_subset.loc[test_idx, id_col_actual_name].values, target_col_actual_name: y_test}
            nan_preds = np.full(len(test_idx), np.nan)
            for model_name_init, run_flag in config.RUN_MODELS.items():
                if run_flag: window_results[f'yhat_{model_name_init.lower()}'] = nan_preds.copy()
            window_models_fitted_this_run = {}

            # === 7: Innermost Loop: Models ===
            for model_name, do_run in config.RUN_MODELS.items():
                if not do_run: continue
                if model_name == 'OLS3H' and (not STATSMODELS_AVAILABLE or not config.RUN_MODELS['OLS3H']): continue
                if model_name.startswith('NN') and not TENSORFLOW_AVAILABLE: continue

                print(f"  -> Trener/Evaluerer: {model_name}...")
                model_start_time = time.time(); fitted_model, preds_oos, preds_is, optimal_hyperparams = None, np.array([]), np.array([]), {}; y_is_target = np.array([])

                current_features = feature_map.get(model_name);
                current_features = [f for f in current_features if f in df_subset.columns] # Ensure features exist
                if not current_features: print(f"    Advarsel: Ingen features for {model_name} i dette vinduet/subset."); continue

                # Get data using the potentially filtered indices
                X_train = df_subset.loc[train_idx, current_features].values
                X_val = df_subset.loc[val_idx, current_features].values if not val_idx.empty else np.empty((0, len(current_features)))
                X_test = df_subset.loc[test_idx, current_features].values
                X_train_val = np.vstack((X_train, X_val)) if not val_idx.empty else X_train

                # Check data validity again after potential filtering
                min_obs_train = max(2, X_train.shape[1] + 1) if model_name == 'OLS3H' else 2
                if X_train.shape[0] < min_obs_train or len(y_train)==0: print(f"    Advarsel: Utilstrekkelig train data ({X_train.shape[0]} obs)."); continue
                if val_idx.empty and needs_val_set(model_name): print(f"    Advarsel: Tomt val set, kan ikke trene {model_name}."); continue
                if not val_idx.empty and (X_val.shape[0] < 2 or len(y_val)==0) and needs_val_set(model_name): print(f"    Advarsel: Utilstrekkelig val data ({X_val.shape[0]} obs)."); continue

                try:
                    # *** Corrected NN Function Call Logic ***
                    if model_name.startswith('NN'):
                        if TENSORFLOW_AVAILABLE:
                            train_function = train_evaluate_nn # Use the generic NN trainer
                            nn_specific_config = config.MODEL_PARAMS.get(model_name, {})
                            if not nn_specific_config: print(f"    FEIL: Mangler config for {model_name}"); continue
                            fitted_model, preds_oos, preds_is, optimal_hyperparams = train_function(
                                X_train, y_train, X_val, y_val, X_test,
                                config.MODEL_PARAMS['NN_SHARED'], # Pass shared NN params
                                nn_specific_config                 # Pass specific NN config (layers etc)
                            )
                            y_is_target = y_train_val # NNs train on train+val after tuning
                        else:
                            continue # Skip NN if TF not available
                    else:
                        # Call specific function for other models
                        train_func_name = f"train_evaluate_{model_name.lower().replace('-', '').replace('+', '_')}"
                        train_function = locals().get(train_func_name)
                        if train_function:
                            model_config_params = config.MODEL_PARAMS.get(model_name, {})
                            if model_name in ['OLS', 'OLS3H']: fitted_model, preds_oos, preds_is, optimal_hyperparams = train_function(X_train_val, y_train_val, X_test, model_config_params); y_is_target = y_train_val
                            elif model_name == 'ENET': fitted_model, preds_oos, preds_is, optimal_hyperparams = train_function(X_train, y_train, X_test, model_config_params); y_is_target = y_train # ENET CV on train only
                            else: fitted_model, preds_oos, preds_is, optimal_hyperparams = train_function(X_train, y_train, X_val, y_val, X_test, model_config_params); y_is_target = y_train_val
                        else: print(f"    FEIL: Treningsfunksjon '{train_func_name}' ikke funnet."); continue

                    # Process results if training was successful
                    if preds_oos is not None and preds_is is not None and len(preds_oos) == len(y_test):
                        # Ensure predictions are finite before calculating metrics
                        finite_oos_mask = np.isfinite(preds_oos)
                        finite_is_mask = np.isfinite(preds_is)

                        preds_oos_finite=preds_oos[finite_oos_mask]; y_test_aligned_oos=y_test[finite_oos_mask]
                        preds_is_finite=preds_is[finite_is_mask]; y_is_target_aligned_is=y_is_target[finite_is_mask]

                        r2_oos=utils.calculate_oos_r2(y_test_aligned_oos, preds_oos_finite) if len(y_test_aligned_oos)>=2 else np.nan
                        r2_is=utils.calculate_oos_r2(y_is_target_aligned_is, preds_is_finite) if len(y_is_target_aligned_is)>=2 else np.nan
                        sharpe_oos=utils.calculate_sharpe_of_predictions(preds_oos_finite) if len(preds_oos_finite)>=2 else np.nan

                        all_metrics[subset][model_name]['oos_r2'].append(r2_oos)
                        all_metrics[subset][model_name]['is_r2_train_val'].append(r2_is)
                        all_metrics[subset][model_name]['oos_sharpe'].append(sharpe_oos)
                        for param_name, param_value in optimal_hyperparams.items(): all_metrics[subset][model_name][f'optim_{param_name}'].append(param_value)

                        pred_col_name=f'yhat_{model_name.lower()}'
                        # Place predictions into results, handling NaNs where needed
                        window_results[pred_col_name].fill(np.nan) # Reset column first
                        if len(preds_oos) == len(window_results[pred_col_name]):
                             window_results[pred_col_name][:] = preds_oos # Assign valid predictions
                        else:
                              print(f"    Advarsel: Lengde mismatch for {model_name} prediksjoner.")

                        if fitted_model is not None: window_models_fitted_this_run[model_name] = fitted_model
                        print(f"    {model_name}: OOS R²={r2_oos:.4f}, IS R²={r2_is:.4f}, Sharpe={sharpe_oos:.3f} ({time.time()-model_start_time:.1f}s)")

                        # --- Calculate VI (if requested and strategy matches) ---
                        if config.CALCULATE_VI and fitted_model is not None and config.MODEL_VI_STRATEGY.get(model_name) == 'per_window':
                            if pd.notna(r2_is):
                                vi_start_time = time.time()
                                # Determine data for VI based on where IS R2 was calculated
                                if model_name == 'ENET':
                                     X_eval_vi_data = X_train; y_eval_vi_data = y_train
                                else:
                                     X_eval_vi_data = X_train_val; y_eval_vi_data = y_is_target

                                if X_eval_vi_data.shape[0] > 0 and y_eval_vi_data.shape[0] > 0:
                                    # Pass optimal hyperparams found in this window
                                    vi_df = utils.calculate_variable_importance(model_name, fitted_model, X_eval_vi_data, y_eval_vi_data, current_features, r2_is, config.VI_METHOD, optimal_hyperparams)
                                    if vi_df is not None and not vi_df.empty: all_vi[subset][model_name].append(vi_df)
                                else: print(f"      Advarsel: Ingen data for VI for {model_name}.")
                            else: print(f"    Advarsel: Hopper over VI for {model_name} pga. NaN IS R2.")
                    else:
                        print(f"    Advarsel: {model_name} returnerte ingen/feil prediksjoner.");
                        all_metrics[subset][model_name]['oos_r2'].append(np.nan); all_metrics[subset][model_name]['is_r2_train_val'].append(np.nan); all_metrics[subset][model_name]['oos_sharpe'].append(np.nan)

                except Exception as e_train:
                    print(f"    !!! KRITISK FEIL under trening/evaluering av {model_name}: {e_train}"); traceback.print_exc();
                    all_metrics[subset][model_name]['oos_r2'].append(np.nan); all_metrics[subset][model_name]['is_r2_train_val'].append(np.nan); all_metrics[subset][model_name]['oos_sharpe'].append(np.nan)

            # End Model Loop
            window_preds_list.append(pd.DataFrame(window_results))
            # Store indices and models from the *last successfully completed* window for last_window VI
            if window_idx == num_windows - 1:
                 last_train_idx=train_idx.copy()
                 last_val_idx=val_idx.copy() if not val_idx.empty else None
                 last_models_fit=window_models_fitted_this_run.copy()
            print(f"-- Vindu {window_num} ({subset}) fullført ({time.time() - window_start_time:.1f}s) --")
        # End Window Loop

        # === 8-10: Post-Window Analysis for the Subset ===
        if not window_preds_list: print(f"\nFEIL: Ingen vindusprediksjoner for '{subset}'."); continue
        print(f"\n--- Analyserer resultater for Subset: {subset} ---")
        results_df_subset=pd.concat(window_preds_list).reset_index(drop=True)

        # Identify prediction columns that actually contain non-NaN data
        prediction_cols_subset = [c for c in results_df_subset.columns if c.startswith('yhat_') and results_df_subset[c].notna().any()]
        if not prediction_cols_subset: print(f"FEIL: Ingen gyldige prediksjonskolonner funnet i results_df for '{subset}'."); continue

        # Calculate Overall OOS R2 (Gu Definition)
        print(f"\n--- Overall OOS R² (Gu-stil) for Subset: {subset} ---")
        y_true_overall=results_df_subset[target_col_actual_name]
        # Check if there are any finite true values
        if not np.any(np.isfinite(y_true_overall)):
             print("  Kan ikke beregne overall OOS R2: Ingen finite y_true verdier.")
        else:
             y_true_finite_overall=y_true_overall[np.isfinite(y_true_overall)]
             ss_tot_overall=np.sum(y_true_finite_overall**2)
             if len(y_true_finite_overall) > 1 and ss_tot_overall > 1e-15:
                 for pred_col in prediction_cols_subset:
                     model_name_oos=pred_col.replace('yhat_', '').upper().replace('_', '-') # Clean model name
                     y_pred_overall=results_df_subset[pred_col]
                     mask_overall=np.isfinite(y_true_overall)&np.isfinite(y_pred_overall)
                     y_t_o=y_true_overall[mask_overall]; y_p_o=y_pred_overall[mask_overall]
                     if len(y_t_o) >= 2:
                          r2_overall_gu=utils.calculate_oos_r2(y_t_o, y_p_o)
                          all_metrics[subset][model_name_oos]['oos_r2_overall_gu']=r2_overall_gu
                          print(f"  {model_name_oos}: {r2_overall_gu:.6f}")
                     else:
                          print(f"  {model_name_oos}: N/A (for få overlappende finite verdier)")
                          all_metrics[subset][model_name_oos]['oos_r2_overall_gu']=np.nan
             else: print("  Kan ikke beregne overall OOS R2 (for få observasjoner eller SS_tot nær null).")

        # Perform Portfolio Analysis (passing VALID prediction cols)
        print(f"\n--- Starter Detaljert Porteføljeanalyse for Subset: {subset} ---")
        decile_tables, hl_risk_tables, long_risk_tables = utils.perform_detailed_portfolio_analysis(
            results_df_subset, df_clean, prediction_cols_subset, # Pass only valid cols
            mcap_orig_col_actual_name, next_ret_col_actual_name,
            config.FILTER_SMALL_CAPS_PORTFOLIO, config.ANNUALIZATION_FACTOR,
            config.BENCHMARK_FILE, config.FF_FACTOR_FILE
            )
        all_portfolios[subset]={'decile_tables': decile_tables, 'hl_risk_tables': hl_risk_tables, 'long_risk_tables': long_risk_tables}

        # Calculate Average/Last Window Variable Importance
        if config.CALCULATE_VI:
            print(f"\n--- Beregner Variabel Viktighet (VI) for Subset: {subset} ---")
            for model_name, do_run in config.RUN_MODELS.items():
                if not do_run: continue
                # Skip models that consistently fail or aren't available
                if model_name=='OLS3H' and (not STATSMODELS_AVAILABLE or not config.RUN_MODELS['OLS3H']): continue
                if model_name.startswith('NN') and not TENSORFLOW_AVAILABLE: continue
                # Skip if no metrics were calculated for this model (implies it never ran successfully)
                if model_name not in all_metrics[subset]: print(f"  Hopper over VI for {model_name} (ingen metrikker funnet)."); continue

                vi_strategy=config.MODEL_VI_STRATEGY.get(model_name);
                current_features_vi=feature_map.get(model_name); current_features_vi=[f for f in current_features_vi if f in df_subset.columns]
                if not current_features_vi: print(f"  Hopper over VI for {model_name} (ingen features funnet)."); continue

                if vi_strategy=='per_window':
                    vi_list_model = all_vi[subset].get(model_name, [])
                    if vi_list_model:
                        try:
                            # Filter out empty DataFrames before concatenating
                            valid_vi_dfs = [df for df in vi_list_model if isinstance(df, pd.DataFrame) and not df.empty]
                            if valid_vi_dfs:
                                avg_vi_df=pd.concat(valid_vi_dfs).groupby('Feature')['Importance'].mean().reset_index()
                                total_avg_importance=avg_vi_df['Importance'].sum()
                                avg_vi_df['Importance']=avg_vi_df['Importance']/total_avg_importance if total_avg_importance > 1e-9 else 0.0
                                all_vi_avg[subset][model_name]=avg_vi_df.sort_values('Importance', ascending=False).reset_index(drop=True)
                                print(f"  VI (Avg/Window) beregnet for {model_name}.")
                            else: print(f"  Ingen gyldige 'per_window' VI dataFrames for {model_name}.")
                        except Exception as e_vi_avg: print(f"  FEIL under VI avg for {model_name}: {e_vi_avg}")
                    else: print(f"  Ingen 'per_window' VI data funnet for {model_name}.")
                elif vi_strategy=='last_window':
                    print(f"  Beregner 'last_window' VI for {model_name}...")
                    if last_train_idx is None or model_name not in last_models_fit: print(f"    Hoppet over {model_name} (mangler data/modell fra siste vindu)."); continue

                    last_model_instance=last_models_fit[model_name]
                    last_is_r2_list=all_metrics[subset][model_name].get('is_r2_train_val', [])
                    last_is_r2 = last_is_r2_list[-1] if last_is_r2_list else np.nan

                    if pd.isna(last_is_r2): print(f"    Advarsel: IS R2 siste vindu NaN for {model_name}, hopper VI."); continue

                    # Get optimal hyperparams from the last window
                    last_optimal_params = {}
                    for k, v_list in all_metrics[subset][model_name].items():
                         if k.startswith('optim_') and v_list:
                             last_optimal_params[k.replace('optim_', '')] = v_list[-1]

                    # Determine data used for IS R2 calculation in the last window
                    if model_name == 'ENET':
                         X_eval_last=df_subset.loc[last_train_idx, current_features_vi].values
                         y_eval_last=df_subset.loc[last_train_idx, target_col_actual_name].values
                    else:
                         last_full_idx=last_train_idx.union(last_val_idx) if last_val_idx is not None else last_train_idx
                         X_eval_last=df_subset.loc[last_full_idx, current_features_vi].values
                         y_eval_last=df_subset.loc[last_full_idx, target_col_actual_name].values

                    # Ensure data is valid before calculating VI
                    if X_eval_last.shape[0] > 0 and y_eval_last.shape[0] > 0:
                         vi_df_last=utils.calculate_variable_importance(
                             model_name, last_model_instance, X_eval_last, y_eval_last,
                             current_features_vi, last_is_r2, config.VI_METHOD, last_optimal_params
                         )
                         if vi_df_last is not None and not vi_df_last.empty:
                             all_vi_avg[subset][model_name]=vi_df_last.sort_values('Importance', ascending=False).reset_index(drop=True)
                             print(f"    VI ({model_name}, siste vindu) beregnet.")
                         else: print(f"    VI beregning siste vindu ({model_name}) mislyktes eller returnerte tom.")
                    else: print(f"    Advarsel: Ingen gyldige evalueringsdata for VI siste vindu ({model_name}).")

        # Generate Summary Table, Plot Complexity, Plot VI
        all_summaries[subset] = utils.create_summary_table(all_metrics[subset], config.ANNUALIZATION_FACTOR)
        utils.plot_time_varying_complexity(all_metrics[subset], config.COMPLEXITY_PARAMS_TO_PLOT)
        if config.CALCULATE_VI and all_vi_avg[subset]:
             print(f"\n--- Plotter Variabel Viktighet for Subset: {subset} ---")
             for model_name, vi_df in all_vi_avg[subset].items():
                 if vi_df is None or vi_df.empty: continue # Skip if no VI data
                 plt.figure(figsize=(10, max(6, min(len(vi_df), config.VI_PLOT_TOP_N) * 0.3)))
                 plot_df = vi_df[vi_df['Importance'] > 1e-6].head(config.VI_PLOT_TOP_N).sort_values(by='Importance', ascending=True)
                 if not plot_df.empty:
                      plt.barh(plot_df['Feature'], plot_df['Importance'])
                      plt.xlabel("Relativ Viktighet (Permutation Importance)")
                      plt.title(f"{model_name} Variable Importance ({subset} - Top {len(plot_df)})")
                      plt.tight_layout(); plt.show()
                 else: print(f"  Ingen VI data å plotte for {model_name} ({subset})."); plt.close()

        # Save Results
        results_to_save={
            'summary_metrics': all_summaries[subset],
            'portfolio_deciles': all_portfolios[subset].get('decile_tables', {}),
            'portfolio_hl_risk': all_portfolios[subset].get('hl_risk_tables', {}),
            'portfolio_long_risk': all_portfolios[subset].get('long_risk_tables', {}),
            'variable_importance_avg': all_vi_avg[subset],
            # Optionally save raw predictions if needed
            # 'raw_predictions': results_df_subset
            }
        utils.save_results(config.OUTPUT_DIR, subset, results_to_save)
        subset_end_time = datetime.datetime.now()
        print(f"\n{'='*30} Subset Fullført: {subset.upper()} (Tid: {subset_end_time - subset_start_time}) {'='*30}")
    # End Subset Loop

    # === Final Reporting ===
    print("\n\n" + "="*35 + " SLUTTSAMMENDRAG " + "="*35)
    r2_final_data = defaultdict(dict)
    for sub in config.SUBSETS_TO_RUN:
        if sub in all_metrics:
            for model, metrics in all_metrics[sub].items():
                # Check if the model actually ran and produced an overall R2
                if 'oos_r2_overall_gu' in metrics:
                     r2_final_data[sub][model] = metrics['oos_r2_overall_gu'] * 100
                else:
                     r2_final_data[sub][model] = np.nan # Mark as NaN if not calculated
    if r2_final_data:
        r2_summary_final = pd.DataFrame.from_dict(r2_final_data, orient='index')
        # Ensure all models from config.RUN_MODELS are columns, even if they didn't run
        model_order_final = [m for m in config.RUN_MODELS if config.RUN_MODELS[m]] # Models intended to run
        all_cols = model_order_final + sorted([m for m in r2_summary_final.columns if m not in model_order_final])
        r2_summary_final = r2_summary_final.reindex(columns=all_cols, fill_value=np.nan)
        r2_summary_final.index.name="Subset"; r2_summary_final.columns.name="Model"
        print("\n--- Tabell 1 Stil: Overall Monthly OOS R² (%) [Gu et al. Def] ---")
        print(r2_summary_final.to_string(float_format=lambda x: f"{x:.3f}" if pd.notna(x) else "N/A", na_rep="N/A"))
        utils.save_results(config.OUTPUT_DIR, "consolidated", {"R2_summary_table1_style": r2_summary_final})
    else: print("\nIngen data for endelig OOS R2-oppsummering.")
    overall_end_time = datetime.datetime.now()
    print(f"\n--- Pipeline Fullført ---"); print(f"--- Sluttidspunkt: {overall_end_time:%Y-%m-%d %H:%M:%S} ---"); print(f"--- Total Kjøretid: {overall_end_time - overall_start_time} ---"); print(f"--- Resultater lagret i: {config.OUTPUT_DIR} ---")
